# DM - PROJECT
## Game of Thrones first and third book analysis
---------

## Data Cleaning


### Imports

In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk import ngrams
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from siuba import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/suru/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/suru/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/suru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Important data saved from the cleaning notebook
-------

In [2]:
%store -r game_path
%store -r swords_path

%store -r game_text_cleaned
%store -r swords_text_cleaned

%store -r game_df_cleaned
%store -r swords_df_cleaned

In [2]:
from siuba import count

def analize_frequency(before_df) -> pd.DataFrame:
    df = before_df.copy()
    total_words = count(df, _.book) #count the total number of words from a book
    word_count = count(df, _.word)  #count the number of occurrences of a word in a book

    # add the new data to the data frame
    df = df.merge(word_count, on='word', how='left')
    df = df.merge(total_words, on='book', how='left')

    # rename the columns accordingly
    df = df.rename(columns={'n_x' : 'word_count', 'n_y' : 'total'})

    # calculate and add the frequency of each word
    df = df.assign(frequency = _.word_count / _.total)

    # rank the word in accordance to their frequency
    df['rank'] = df.groupby('book')['word_count'].rank(method='dense', ascending=True)


    return df

In [116]:
# add the name of the book in each data frame
game_df = game_df.assign(book = 'A Game of Thrones')
sword_df = sword_df.assign(book = 'Storm of Swords')

# perform word analysis
game_df = analize_frequency(game_df)
sword_df = analize_frequency(sword_df)

game_df.head(15000)

,word,book,word_count,total,frequency,rank
0,book,A Game of Thrones,79,594862,0.000133,79.0
1,one,A Game of Thrones,3045,594862,0.005119,538.0
2,song,A Game of Thrones,176,594862,0.000296,176.0
3,ice,A Game of Thrones,258,594862,0.000434,250.0
4,fire,A Game of Thrones,629,594862,0.001057,425.0
...,...,...,...,...,...,...
14995,lannisters,A Game of Thrones,291,594862,0.000489,275.0
14996,proud,A Game of Thrones,100,594862,0.000168,100.0
14997,”,A Game of Thrones,27705,594862,0.046574,544.0
14998,jon,A Game of Thrones,2532,594862,0.004256,534.0


In [117]:
from tidytext import bind_tf_idf

def td_idf_analysis(before_df) -> pd.DataFrame:
    df = before_df.copy()
    df = bind_tf_idf(df, 'word', 'book', 'word_count')
    df.sort_values(by=['tf_idf'], inplace=True, ascending=True)


    return df

In [118]:
game_df = td_idf_analysis(game_df)
game_df.head(1500000)

C:\University\year3\Sem2\DM\Project\Data-Mining\venv\lib\site-packages\tidytext.py:102: UserWarning: A value for tf_idf is negative: 
Input should have exactly one row per document-term combination.
  warnings.warn(


,word,book,word_count,total,frequency,rank,tf,idf,tf_idf
297430,“,A Game of Thrones,27906,594862,0.046912,545.0,1.274453e-05,-10.236597,-0.00013
456714,“,A Game of Thrones,27906,594862,0.046912,545.0,1.274453e-05,-10.236597,-0.00013
456696,“,A Game of Thrones,27906,594862,0.046912,545.0,1.274453e-05,-10.236597,-0.00013
456687,“,A Game of Thrones,27906,594862,0.046912,545.0,1.274453e-05,-10.236597,-0.00013
456682,“,A Game of Thrones,27906,594862,0.046912,545.0,1.274453e-05,-10.236597,-0.00013
...,...,...,...,...,...,...,...,...,...
254625,were—rode,A Game of Thrones,1,594862,0.000002,1.0,4.566949e-10,0.000000,0.00000
418966,jyanna,A Game of Thrones,1,594862,0.000002,1.0,4.566949e-10,0.000000,0.00000
254631,hundredodd,A Game of Thrones,1,594862,0.000002,1.0,4.566949e-10,0.000000,0.00000
254600,eightythree,A Game of Thrones,1,594862,0.000002,1.0,4.566949e-10,0.000000,0.00000
